In [2]:
import os
import csv
import pandas as pd
import numpy as np

In [3]:
os.chdir('/Users/miatsai/Downloads')

In [4]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]
for folder_name in os.listdir("./Competition_data"):
    dataset_names.append(folder_name)
    X_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/X_train.csv",header=0))
    y_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/y_train.csv",header=0))
    X_tests.append(pd.read_csv(f"./Competition_data/{folder_name}/X_test.csv",header=0))

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


In [6]:
models = []
scalers = [] #測試的參數
poly_transformers = []

In [15]:
for i in range(len(dataset_names)):
    poly = PolynomialFeatures(degree=3, interaction_only=True)
    X_train_poly = poly.fit_transform(X_trains[i])

    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_train_poly,
        y_trains[i],
        test_size=0.2,
        random_state=42
    )

    scaler = StandardScaler()
    tmp_X_train_scaled = scaler.fit_transform(tmp_X_train)
    tmp_X_test_scaled = scaler.transform(tmp_X_test)

 

    for c in [0.001, 0.01, 0.1, 1, 10, 100]:  #嘗試的參數
        model = LogisticRegression(
            C = c,
            max_iter=1000,
            class_weight= 'balanced',
            random_state=42
        )
        model.fit(tmp_X_train_scaled, tmp_y_train)
        tmp_y_prob = model.predict_proba(tmp_X_test_scaled)[:, 1]
        auc = roc_auc_score(tmp_y_test, tmp_y_prob)

      

    models.append(model)
    scalers.append(scaler) 
    poly_transformers.append(poly)

/Users/miatsai/Library/Python/3.12/lib/python/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/miatsai/Library/Python/3.12/lib/python/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/miatsai/Library/Python/3.12/lib/python/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/miatsai/Library/Python/3.12/lib/python/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when 

In [16]:
tmp_X_train_scaled.shape

(368, 93)

In [17]:
y_predicts = []
for i in range(len(dataset_names)):
    X_test_poly = poly_transformers[i].transform(X_tests[i])
    X_test_scaled = scalers[i].transform(X_test_poly)

    y_predict_proba = models[i].predict_proba(X_test_scaled)[:, 1]
    df = pd.DataFrame(y_predict_proba, columns = ['y_predict_proba'])
    y_predicts.append(df)

In [14]:
for idx, dataset_name in enumerate(dataset_names):
    df = y_predicts[idx]
    df.to_csv(f'./Competition_data/{dataset_name}/y_predict.csv', index=False, header=True)
